In [6]:
!pip install pandas
!pip install csv


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\lisaa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\lisaa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [67]:
def weighting_educ(column_mf, column_level):
    #Creating the observed data
    possible_levels = {"HS-grad", "Some-college", "Assoc-acdm", "Assoc-voc", "Bachelors", "Masters", "Prof-school", "Doctorate"}
    p_obs_male_dict = {"8":0, "11":0, "HS-grad": 0, "Some-college": 0, "assoc": 0, "Bachelors": 0, "Masters": 0, "Prof-school": 0, "Doctorate": 0}
    p_obs_female_dict = {"8":0, "11":0, "HS-grad": 0, "Some-college": 0, "assoc": 0, "Bachelors": 0, "Masters": 0, "Prof-school": 0, "Doctorate": 0}
    female_ob = 0
    male_ob = 0
    for i,(level,gender) in enumerate(zip(column_level, column_mf)):
        #Counting all instances per category
        if not level in possible_levels:
            try:
                if level == "Preschool":
                    if gender == "Male":
                        p_obs_male_dict["8"] += 1
                    else:
                        p_obs_female_dict["8"] += 1
                elif int(level[0]) <= 8:
                    if gender == "Male":
                        p_obs_male_dict["8"] += 1
                    else:
                        p_obs_female_dict["8"] += 1
            except:
                pass
            else:
                if gender == "Male":
                    p_obs_male_dict["11"] += 1
                else:
                    p_obs_female_dict["11"] += 1  
        elif level in {"Assoc-acdm", "Assoc-voc"}:
            if gender == "Male":
                    p_obs_male_dict["assoc"] += 1
            else:
                p_obs_female_dict["assoc"] += 1
        else:
            if gender == "Male":
                p_obs_male_dict[level] += 1
            else:
                p_obs_female_dict[level] += 1
        #Counting the amount of men and women
        if gender == "Male":
            male_ob += 1
        else:
            female_ob += 1
    #Changing the dicts used to count into lists
    total = male_ob + female_ob
    p_obs_male = list(p_obs_male_dict.values())
    p_obs_male = [x  /total for x in p_obs_male]
    p_obs_female = list(p_obs_female_dict.values())
    p_obs_female = [x  / total for x in p_obs_female]
    
    #Calculating expected data using data from:
    #https://www.census.gov/data/tables/2022/demo/educational-attainment/cps-detailed-tables.html
    total =  226274
    p_exp_male = [x /total for x in [4041,	6303,	33081,	16085,	10616,	25192,	10156,	1860,	2644] ]
    p_exp_female = [x /total for x in [3895,	5695,	31383,	16900,	13057,	27853,	13725,	1584,	2203] ]
    
    #calculating the weights by deviding the expected by the observed. The output is in order of the dicts on the first lines of the function.
    weights_male = []
    weights_female = []
    for obs_m, exp_m, obs_f, exp_f in zip(p_obs_male, p_exp_male, p_obs_female, p_exp_female):
        weights_male.append(exp_m / obs_m)
        weights_female.append(exp_f / obs_f)
    
    return p_obs_male_dict.keys(), weights_male, weights_female
    

In [72]:
#Reweighing for marital status
#https://www.census.gov/library/visualizations/interactive/marital-status-in-united-states.html
def weighting_maritalstatus(column_mf, column_mar):
    #Creating the observed data
    possible_status = {"Never-married", "Married-civ-spouse", "Married-spouse-absent", "Separated", "Divorced", "Widowed"}
    p_obs_male_dict = {"Never-married": 0, "Married": 0, "Separated": 0, "Divorced": 0, "Widowed": 0}
    p_obs_female_dict =  {"Never-married": 0, "Married": 0, "Separated": 0, "Divorced": 0, "Widowed": 0}
    female_ob = 0
    male_ob = 0
    for i,(status,gender) in enumerate(zip(column_mar, column_mf)):
        #Counting all instances per category
        if not status in possible_status:
            pass
        else:
            if status == "Married-civ-spouse" or status == "Married-spouse-absent":
                status = "Married"
            
            if gender == "Male":
                p_obs_male_dict[status] += 1
            else:
                p_obs_female_dict[status] += 1
        #Counting the amount of men and women
        if gender == "Male":
            male_ob += 1
        else:
            female_ob += 1
    total = male_ob + female_ob
    #Changing the dicts used to count into lists
    p_obs_male = list(p_obs_male_dict.values())
    p_obs_male = [x  /total for x in p_obs_male]
    p_obs_female = list(p_obs_female_dict.values())
    p_obs_female = [x  / total for x in p_obs_female]

    #Calculating expected data using data from:
    male = 129973543
    female = 136348759
    total = male + female 
    #The amount of people per group divide by total for expected, done the same as observed on different data.
    p_exp_male = [x /total for x in [48052364,64057452,2071170,12365418, 3427139] ]
    p_exp_female = [x /total for x in [41880024,63122995,2963251,16553544,11828945]]
    
    #calculating the weights by deviding the expected by the observed. The output is in order of the dicts on the first lines of the function.
    weights_male = []
    weights_female = []
    for obs_m, exp_m, obs_f, exp_f in zip(p_obs_male, p_exp_male, p_obs_female, p_exp_female):
        weights_male.append(exp_m / obs_m)
        weights_female.append(exp_f / obs_f)
    
    return p_obs_male_dict.keys(), weights_male, weights_female
    

In [73]:
# applying weights
import csv
gender = []
educ = []
status = []
with open('adult.csv', newline='') as data:
    spamreader = csv.reader(data, delimiter=' ')
    for row in spamreader:
        row = row[0].split(",")
        #[1:-1] because the words look like this: "'word'"
        educ.append(row[3][1:-1])
        gender.append(row[9][1:-1])
        status.append(row[5][1:-1])
pos_level, male_educ, female_educ = weighting_educ(gender, educ)
pos_status, male_mar, female_mar = weighting_maritalstatus(gender, status)
print(pos_level, "\n", male_educ,"\n" , female_educ, "\n", pos_status, "\n", male_mar, "\n", female_mar)

dict_keys(['8', '11', 'HS-grad', 'Some-college', 'assoc', 'Bachelors', 'Masters', 'Prof-school', 'Doctorate']) 
 [0.2269792427365061, 0.3093568656412059, 0.6694595918070898, 0.5161013981243859, 0.9998019359098849, 0.9703584627146346, 1.2312560881987997, 0.5530240269659172, 1.163563571096275] 
 [0.47621976555833023, 0.6203936279326979, 1.3322053589817218, 0.8667132892975913, 2.040141036814523, 2.475720131227805, 3.684885360226981, 2.4776717415707235, 3.6863151712957185] 
 dict_keys(['Never-married', 'Married', 'Separated', 'Divorced', 'Widowed']) 
 [0.9930934758755159, 0.5787769230260589, 0.6427218448608766, 0.8536773849373384, 2.494169552848252] 
 [1.0741509227484345, 4.144875173611973, 0.574172801665185, 0.7574571502995334, 1.7530552344190702]
